# 7.7 Logical parameters

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Although parameters normally represent numeric values, they can optionally be used
to stand for true-false values or for character strings.

The current version of AMPL does not support a full-fledged "logical" type of parameter
that would stand for only the values true and false, but a parameter of type binary
may be used to the same effect. As an illustration, we describe an application of the preceding
inventory example to consumer goods. Certain products in each week may be
specially promoted, in which case they require a higher inventory fraction. Using parameters
of type binary, we can represent this situation by the following declarations:
```
param fr_reg > 0;                # regular inventory fraction
param fr_pro > fr_reg;           # fraction for promoted items
param promote {PROD,1..T+1} binary;
param market {PROD,1..T+1} >= 0;
```

The binary parameters `promote[p,t]` are 0 when there is no promotion, and 1 when
there is a promotion. Thus we can define the minimum-inventory parameters by use of
an if-then-else expression as follows:
```
param mininv {p in PROD, t in 0..T} =
  (if promote[p,t] = 1 then fr_pro else fr_reg)
     * market[p,t+1];
```
We can also say the same thing more concisely:
```
param mininv {p in PROD, t in 0..T} =
  (if promote[p,t] then fr_pro else fr_reg) * market[p,t+1];
```
When an arithmetic expression like `promote[p,t]` appears where a logical expression
is required, AMPL interprets any nonzero value as true, and zero as false. You do need to
exercise a little caution to avoid being tripped up by this implicit conversion. For
example , in [Section 7.4](../07/7_4_restrictions_on_parameters.ipynb) we used the expression
```
if t = 1 then 0 else cumulative_market[p,t-1]
```
If you accidentally write
```
if t then 0 else cumulative_market[p,t-1] # DIFFERENT
```
it's perfectly legal, but it doesn't mean what you intended.
